In [16]:
import numpy as np

#load the labeled segments.
home =r'/home/naresh-1/workspace/machinelrn/data/gis/'
srcshpfile = home + r'out/usa.shp'
destdir = home+'out/'

inputfile = 'usa.prj.lbl.txn.int.txt'
segments = np.loadtxt(destdir+inputfile,comments='#',dtype=object)
segments.shape
endpoints = segments[:,1:5]
endpoints.shape
endpoints = endpoints.astype(int)
#merge both x1,x2 --> get unique x-values
xuniq = sorted(list(np.unique(np.vstack((endpoints[0:,0:1],endpoints[0:,2:3])))))
print endpoints.shape, len(xuniq)

#find unique x-values.

(56282, 4) 55149


In [17]:
#maximum and minimum for the map.
endpoints.max(axis=0), endpoints.min(axis=0), endpoints.shape

(array([1258460509, 2180444978, 1258468012, 2180444978]),
 array([ 615227493, 1834173030,  615232153, 1834173030]),
 (56282, 4))

In [19]:
#arrange segments (x1,y1)--(x2,y2) such that x1<x2.
#remove duplicates
uniqseg = []
import time
st = time.time()
for seg in endpoints:
    x1,y1,x2,y2 = seg
    if x1 < x2:
        oseg = [x1,y1,x2,y2]
    else:
        oseg = [x2,y2,x1,y1]
    if oseg not in uniqseg:
        uniqseg.append(oseg)

print endpoints.shape, len(uniqseg)
print time.time()-st

(56282, 4) 56282
92.7873361111


In [29]:
#split segments.
splits = []

for segment in uniqseg:
    x1,y1,x2,y2 = segment
    li = xuniq.index(x1)
    hi = xuniq.index(x2)
    if hi-li < 2:
        splits.append(segment)
    else:
        yc = y1
        for i in range(li,hi):
            xc = xuniq[i]
            xn = xuniq[i+1]
            if xn < x2:
                y=float(y2-y1)/(x2-x1)*(xn-x1)+y1
                yn=int(round(y))
                splits.append([xc,yc,xn,yn])
                yc=yn
            elif xn == x2:splits.append([xc,yc,xn,y2])
print len(uniqseg),len(splits)

56282 627084


In [30]:
print len(xuniq), len(uniqseg),len(splits),len(splits)/len(uniqseg)

55149 56282 627084 11


In [31]:
xdict={}##now add all split lines to a dictionary
for i in range(len(xuniq)-1):
    xl=xuniq[i]
    xh=xuniq[i+1]
    xdict[xl]=[]
xdict[xh]=[] #add last ite,
for i in range(len(splits)):
    x1=splits[i][0]
    y1=splits[i][1]
    x2=splits[i][2]
    y2=splits[i][3]
    if y1<y2:
        #xdict[x1].append((y1,y2))
        xdict[x1]+=[y1,y2]
    else:
        xdict[x1]+=[y2,y1]

In [32]:
for key in xdict.keys()[0:10]:
    print xdict[key]
    xdict[key]= list(set(xdict[key]))

[2109973028, 2110019235, 2053413293, 2053413492]
[2173905348, 2173905365, 1881964331, 1882002491]
[2058716611, 2058719133, 2057337166, 2057341003, 2086492960, 2086494262, 2086437037, 2086457721, 2086434671, 2086437037, 2126242682, 2126243321, 2127356562, 2127358278, 2128100725, 2128102553, 2122772973, 2122774894, 2124011696, 2124012912]
[1845212224, 1845215506, 1910317497, 1910320573, 1910764886, 1910793870, 1907401746, 1907457647, 1907761794, 1907795341, 2057764242, 2057773217]
[2173901954, 2173901972, 1895015439, 1895019841]
[2020510228, 2020510229, 2019495131, 2019496895, 2019053917, 2019056206, 2019991681, 2019993682, 2023259324, 2023261532, 2023402210, 2023403424, 2023799564, 2023803285, 2024715380, 2024718924, 2014687216, 2014691377, 2014677708, 2014683049, 2014605596, 2014605817, 2014574307, 2014577082, 2014451707, 2014455200, 2013844145, 2013844639, 2015864470, 2015867244, 2015632565, 2015633001, 2009518081, 2009521966, 2008483912, 2008487390, 2008627088, 2008629381, 2009862762

In [27]:
len(xdict)

55149

In [23]:
size=[]
for x,ylist in xdict.items():
    size +=[len(ylist)]
max(size),min(size), sum(size)

(58, 0, 627084)

In [24]:
d={}
d[2]=[(1,2),(3,3),(1,2),(3,3)]
d[3]=[3,4,4,4,5]
for k in d.keys():
    d[k] = list(set(d[k]))
print d

{2: [(1, 2), (3, 3)], 3: [3, 4, 5]}
